In [ ]:
import napari
import os
from tqdm import tqdm
import numpy as np
import glob2 as glob
import zarr
import json
import skimage.io as io
import pandas as pd
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

# # set parameters
root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/killi_tracker/" #"E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\\killi_tracker\\"
project_name = "240219_LCP1_93hpf_to_127hpf"
image_dir = os.path.join(root, "built_data", project_name, "")
config_name = "tracking_cell.txt"# 
tracking_folder = config_name.replace(".txt", "")
tracking_folder = tracking_folder.replace(".toml", "")

tracking_directory = os.path.join(root, "built_data", "tracking", project_name, tracking_folder)
snip_directory = os.path.join(root, "built_data", "shape_snips", project_name, tracking_folder, "class0")

# load metadata
metadata_file_path = os.path.join(root, "metadata", project_name, "metadata.json")
f = open(metadata_file_path)
metadata = json.load(f)
scale_vec = np.asarray([metadata["ProbPhysicalSizeZ"], metadata["ProbPhysicalSizeY"], metadata["ProbPhysicalSizeX"]])

## Load tracking DF

In [ ]:
# load tracking data
tracks_df = pd.read_csv(os.path.join(tracking_directory, "tracks_cleaned.csv"))
tracks_df.head()
tracks_df_clean = tracks_df.loc[(~tracks_df["fragment_flag"]) & (tracks_df["avg_disp_um"] > 2.2), :]

## Load sphere coordinate DF

In [ ]:
# load tracking data
sphere_df = pd.read_csv(os.path.join(tracking_directory, "sphere_df_smoothed.csv"))
sphere_df.tail()
si = np.argsort(sphere_df["t"])
sphere_df = sphere_df.loc[si, :]
sphere_df.reset_index(inplace=True)
sphere_df.head()

In [ ]:
np.min(sphere_df["rm"])

## 3D visuals

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def ms(x, y, z, radius, resolution=20):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return (X, Y, Z)

def ms_T(x, y, z, radius, resolution=20):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    v, u = np.mgrid[0:np.pi:resolution*1j, 0:2*np.pi:resolution*2j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return (X, Y, Z)

In [ ]:
from alphashape import alphashape

def generate_sphere_mesh(X, Y, Z):
    
    xyz = np.concatenate((X[:, np.newaxis], Y[:, np.newaxis], Z[:, np.newaxis]), axis=1)
    xyz_min = np.min(xyz, axis=0)
    xyz_norm = xyz - xyz_min
    xyz_max = np.max(xyz_norm, axis=0) 
    xyz_norm = xyz_norm / xyz_max
    sphere_hull = alphashape(xyz_norm, alpha)
    
    tri_points = sphere_hull.vertices[sphere_hull.faces]
    tri_points = np.multiply(tri_points, xyz_max)
    tri_points = tri_points + xyz_min
    #extract the lists of x, y, z coordinates of the triangle vertices and connect them by a line
    Xe = []
    Ye = []
    Ze = []
    for T in tri_points:
        Xe.extend([T[k%3][0] for k in range(4)]+[ None])
        Ye.extend([T[k%3][1] for k in range(4)]+[ None])
        Ze.extend([T[k%3][2] for k in range(4)]+[ None])
        
    return Ze, Ye, Xe
    
       
#define the trace for triangle sides
# fig = go.Figure()
# lines = go.Scatter3d(
#                    x=Xe,
#                    y=Ye,
#                    z=Ze,
#                    mode='lines',
#                    name='',
#                    line=dict(color= 'rgb(70,70,70)', width=1)) 

# fig.add_trace(go.Mesh3d(x=plot_hull.vertices[:, 0], y=plot_hull.vertices[:, 1],z=plot_hull.vertices[:, 2], 
#                         opacity=1,
#                                i=plot_hull.faces[:, 0], j=plot_hull.faces[:, 1], k=plot_hull.faces[:, 2]))


In [ ]:
from tqdm import tqdm
import math 

plotly_repmap = np.asarray(px.colors.qualitative.Plotly*100)
tracks_df_clean["plot_color"] = plotly_repmap[tracks_df_clean["track_id"]]

mem = 25
max_time = sphere_df.shape[0]# np.max(tracks_df["t"])
figure_root = os.path.join(root, "figures", project_name, tracking_folder)
if not os.path.isdir(figure_root):
    os.makedirs(figure_root)
    
rotate_plot = True
rot_period = 100
angle_vec = np.linspace(0, 2*np.pi, rot_period)
    
track_path_3d = os.path.join(figure_root, "track_plots_3d")
if not os.path.isdir(track_path_3d):
    os.makedirs(track_path_3d)

radius_static = 540

for time_id in tqdm(range(max_time)): #range(max_time)):
    tracks_df_ft = tracks_df_clean.loc[tracks_df_clean["t"]==time_id]
    tracks_df_ft_mem = tracks_df_clean.loc[(tracks_df_clean["t"]<=time_id) & (tracks_df["t"]>time_id-mem) ]
    x_pns_surface, y_pns_surface, z_pns_surface = ms(sphere_df.loc[time_id, "XM"], 
                                                      sphere_df.loc[time_id, "YM"], 
                                                      sphere_df.loc[time_id, "ZM"],radius_static)

    x_alt, y_alt, z_alt = ms_T(sphere_df.loc[time_id, "XM"], 
                                                      sphere_df.loc[time_id, "YM"], 
                                                      sphere_df.loc[time_id, "ZM"],radius_static)

    
    point_array = tracks_df_ft.loc[:, ["xum", "yum", "zum"]].to_numpy()
    point_ids = tracks_df_ft.loc[:, ["track_id"]].to_numpy().flatten()
    point_colors = tracks_df_ft.loc[:, ["plot_color"]].to_numpy().flatten()
    track_array = tracks_df_ft_mem.loc[:, ["xum", "yum", "zum"]].to_numpy()
    track_ids = tracks_df_ft_mem.loc[:, ["track_id"]].to_numpy().flatten()
    track_colors = tracks_df_ft_mem.loc[:, ["plot_color"]].to_numpy().flatten()
    
    X = x_pns_surface.flatten()
    Y = y_pns_surface.flatten()
    Z = z_pns_surface.flatten()
    
    Xa = x_alt.flatten()
    Ya = y_alt.flatten()
    Za = z_alt.flatten()
    
#     Ze, Ye, Xe = generate_sphere_mesh(X, Y, Z)

    extant_ids = np.unique(track_ids)

    fig = go.Figure()
    fig.add_trace(go.Surface(x=x_pns_surface, y=y_pns_surface, z=z_pns_surface, opacity=0.7, 
                             colorscale="Blues"))#, lighting=dict(ambient=0.5)))
    
    fig.update_traces(showscale=False)

    lines = fig.add_trace(go.Scatter3d(
                   x=X,
                   y=Y,
                   z=Z,
                   mode='lines',
                   name='',
                   line=dict(color= 'rgba(70,70,70,1)', width=.5)))
    

    lines2 = fig.add_trace(go.Scatter3d(
                   x=Xa,
                   y=Ya,
                   z=Za,
                   mode='lines',
                   name='',
                   line=dict(color= 'rgba(70,70,70,1)', width=.5)))

    for t in range(len(extant_ids)-1):
        tid = extant_ids[t]
        point_ft = point_ids==tid
        track_ft = track_ids==tid

        if np.sum(point_ft) > 0:
            fig.add_trace(go.Scatter3d(x=point_array[point_ft, 0], y=point_array[point_ft, 1], z=point_array[point_ft, 2], 
                                       mode="markers", showlegend=False, marker=dict(color=point_colors[point_ft], size=6)))

        fig.add_trace(go.Scatter3d(x=track_array[track_ft, 0], y=track_array[track_ft, 1], z=track_array[track_ft, 2], 
                                   mode="lines", showlegend=False, line=dict(color=track_colors[track_ft])))

    fig.update_layout(scene = dict(
                        xaxis_title='x (um)',
                        yaxis_title='y (um)',
                        zaxis_title='z (um)', 
                     xaxis = dict(range=[0,1300],),
                     yaxis = dict(range=[0,1300],),
                     zaxis = dict(range=[-800,500],),))
    
    eye_vec = np.asarray([1, 1, 2])*0.85
    
    if rotate_plot:
        ind = time_id % len(angle_vec)
        angle = angle_vec[ind]
        x = math.cos(angle)
        y = math.sin(angle)
        mag = np.sqrt(x**2 + y**2)
        fac = 0.85 / mag
        eye_vec = np.asarray([x*fac, y*fac, eye_vec[2]])

    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=eye_vec[0], y=eye_vec[1], z=eye_vec[2])
    )

    fig.update_layout(scene_camera=camera,
                      template="plotly_dark")
    
    fig.write_image(os.path.join(track_path_3d, f'frame3d_t{time_id:04}.png'), scale=2)

fig.show()

In [ ]:
np.max(np.abs(np.diff(Xa)))

## 2D visuals

In [ ]:
from astropy.coordinates import cartesian_to_spherical

time_id = 700

tracks_df_ft = tracks_df.loc[tracks_df["t"]==time_id]


fig = px.scatter(tracks_df_ft, x="theta", y="phi")
fig.show()

In [ ]:
mem = 25
max_time = sphere_df.shape[0]# np.max(tracks_df["t"])
    
 
plotly_repmap = np.asarray(px.colors.qualitative.Plotly*100)
tracks_df_clean["plot_color"] = plotly_repmap[tracks_df_clean["track_id"]]

big_jump_thresh = 50

track_path_2d = os.path.join(figure_root, "track_plots_2d")
if not os.path.isdir(track_path_2d):
    os.makedirs(track_path_2d)


for time_id in tqdm(range(max_time)): #range(max_time)):
    tracks_df_ft = tracks_df_clean.loc[tracks_df_clean["t"]==time_id]
    tracks_df_ft_mem = tracks_df_clean.loc[(tracks_df_clean["t"]<=time_id) & (tracks_df["t"]>time_id-mem) ]

    
    point_array = tracks_df_ft.loc[:, ["theta", "phi"]].to_numpy() / np.pi * 180
    point_ids = tracks_df_ft.loc[:, ["track_id"]].to_numpy().flatten()
    point_colors = tracks_df_ft.loc[:, ["plot_color"]].to_numpy().flatten()
    track_array = tracks_df_ft_mem.loc[:, ["theta", "phi"]].to_numpy() / np.pi * 180
    track_ids = tracks_df_ft_mem.loc[:, ["track_id"]].to_numpy().flatten()
    track_colors = tracks_df_ft_mem.loc[:, ["plot_color"]].to_numpy().flatten()
    
    extant_ids = np.unique(track_ids)

    fig = go.Figure()
#     fig.add_trace(go.Surface(x=x_pns_surface, y=y_pns_surface, z=z_pns_surface, opacity=0.7, 
#                              colorscale="Blues"))#, lighting=dict(ambient=0.5)))
    
    fig.update_traces(showscale=False)

    for t in range(len(extant_ids)-1):
        tid = extant_ids[t]
        point_ft = point_ids==tid
        track_ft = track_ids==tid

        x_vec = track_array[track_ft, 0]
        y_vec = track_array[track_ft, 1]
        if len(x_vec) > 1:
            xj_ind_vec = (np.where(np.abs(np.diff(x_vec)) > big_jump_thresh)[0] + 1).tolist()
            yj_ind_vec = (np.where(np.abs(np.diff(y_vec)) > big_jump_thresh)[0] + 1).tolist()
        else:
            xj_ind_vec = []
            yj_ind_vec = []
            
        jump_indices = np.unique([0] + xj_ind_vec + yj_ind_vec + [len(x_vec)])
        
        
        for j in range(len(jump_indices)-2, len(jump_indices)-1):
            start_i = jump_indices[j]
            stop_i = jump_indices[j+1]
            fig.add_trace(go.Scatter(x=x_vec[start_i:stop_i], y=y_vec[start_i:stop_i], 
                                       mode="lines", showlegend=False, line=dict(color=track_colors[track_ft][0])))
        
        if np.sum(point_ft) > 0:
            fig.add_trace(go.Scatter(x=point_array[point_ft, 0], y=point_array[point_ft, 1],
                                       mode="markers", showlegend=False, marker=dict(color=point_colors[point_ft], 
                                                                    size=8, line=dict(color="SlateGray", width=1))))

        

    fig.update_layout(
                        xaxis_title='longitude (degrees)',
                        yaxis_title='latitude (degrees)', 
                        yaxis_range=[0, 90],
                        xaxis_range=[0, 360])
    
    
    fig.write_image(os.path.join(track_path_2d, f'frame3d_t{time_id:04}.png'), scale=2)

fig.show()

In [ ]:
tid = extant_ids[13]
point_ft = point_ids==tid
track_ft = track_ids==tid

x_vec = track_array[track_ft, 0]
y_vec = track_array[track_ft, 1]
if len(x_vec) > 1:
    xj_ind_vec = np.where(np.abs(np.diff(x_vec)) > big_jump_thresh)[0].tolist()
    yj_ind_vec = np.where(np.abs(np.diff(y_vec)) > big_jump_thresh)[0].tolist()
else:
    xj_ind_vec = []
    yj_ind_vec = []
    
print(x_vec)

In [ ]:
jump_indices = np.unique([0] + xj_ind_vec + yj_ind_vec + [len(x_vec)])
jump_indices

In [ ]:
from tqdm import tqdm
dt = 90

window_size_vec = [5, 25]

for window_size in window_size_vec:
    w_str = str(window_size)
    c_time = dt*(window_size-1)
    buffer_size = window_size // 2
    track_index = np.unique(tracks_df["track_id"])

    for t, track_id in enumerate(tqdm(track_index)):
        track_bool = tracks_df["track_id"] == track_id

        if np.sum(track_bool) >= window_size:
            xyz = tracks_df.loc[track_bool, ["x", "y", "z"]].to_numpy()

            # get contour length
            d1 = np.sqrt(np.sum(np.diff(xyz, 1, axis=0)**2, axis=1))
            d1_cs = np.cumsum(d1)
            cs1 = d1_cs[window_size-2:]
            cs2 = np.concatenate(([0], d1_cs[:-window_size+1]))
            contour_r = cs1 - cs2

            # get net displacement
            net_r = np.sqrt(np.sum((xyz[window_size-1:, :] - xyz[:-window_size+1, :])**2, axis=1))

            # assign to tracks df
            to_indices = np.where(track_bool)[0]
            to_indices = to_indices[buffer_size:-buffer_size]
            tracks_df.loc[to_indices, "speed_" + w_str] = contour_r / c_time
            tracks_df.loc[to_indices, "speed_net_" + w_str] = net_r / c_time
            tracks_df.loc[to_indices, "coherence_" + w_str] = net_r / contour_r


        

In [ ]:
import plotly.express as px

track_id = 101
fig = px.scatter(tracks_df.loc[tracks_df["track_id"]==track_id], x="t", y="speed_5", color="t", template="plotly")
fig.show()

In [ ]:
fig = px.density_contour(tracks_df, x="speed_25", y="speed_net_25")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()

In [ ]:
fig = px.density_contour(tracks_df, x="t", y="coherence_25")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()

In [ ]:
shape_df = pd.read_csv(os.path.join(tracking_directory, "cell_shape_df.csv"))
shape_df.head()

In [ ]:
fig = px.density_contour(shape_df, x="t", y="circularity")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()

In [ ]:
master_df = tracks_df.merge(shape_df, on=["track_id", "t"], how="left")

In [ ]:
master_df["time_id"] = np.floor(master_df["t"].to_numpy()/350).astype(int)

fig = px.density_contour(master_df, x="complexity", y="speed_25", color="time_id", template="plotly")
# fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()

In [ ]:
fig = px.density_contour(master_df, x="circularity", y="speed_25")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()